In [1]:
import torch
import torch.nn as nn
import numpy as np

# 均方误差

In [2]:
y = torch.tensor([1., 1., 1., 1.])
y_pre1 = torch.tensor([1, 1, 1, 1])
y_pre2 = torch.tensor([2, 2, 2, 2])
y_pre3 = torch.tensor([3, 3, 3, 3])
criterion = nn.MSELoss()
y_pre_li = [y_pre1, y_pre2, y_pre3]
for i in range(len(y_pre_li)):
    # 预测值必须只有一个维度，用这个函数扁平化values.squeeze(1)
    output = criterion(y_pre_li[i], y)
    print('第{}个预测值和实际值的损失值：{}'.format(i+1, output))

第1个预测值和实际值的损失值：0.0
第2个预测值和实际值的损失值：1.0
第3个预测值和实际值的损失值：4.0


In [3]:
# 算法原理
torch.mean((y - y_pre3)**2)

tensor(4.)

# 二分类交叉熵

In [4]:
# BCELoss, 预测值经过sigmoid后转换成0~1之间，标签则是0或者1
y = torch.Tensor([0,0,0,0,0,1,1,1,1,1])
y_pre1 = torch.Tensor([0.1,0.1,0.2,0.2,0.8,1.5,1.2,2.2,3.8,1.8])
y_pre2= torch.Tensor([2.2,2.2,2.2,2.2,2.2,0.8,0.8,0.8,0.8,0.8])
y_pre3= torch.Tensor([0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5])
# 预测值要加一层sigmoid的输出
y_pre_li = [nn.Sigmoid()(y_pre1),nn.Sigmoid()(y_pre2),nn.Sigmoid()(y_pre3)]
criterion = nn.BCELoss(reduction='mean')
for i in range(len(y_pre_li)):
    output = criterion(y_pre_li[i],y)
    print('第{}个预测值和实际值的损失值：{}'.format(i, output))

第0个预测值和实际值的损失值：0.5001052618026733
第1个预测值和实际值的损失值：1.3380919694900513
第2个预测值和实际值的损失值：0.7240769267082214


$\frac{-1}{n}\sum_{n}^{i}(y[i]*log(y_{pre}[i]))+(1-y[i])*log(1-y_{pre}[i])$

In [5]:
# 算法
output = -1*torch.mean(y*torch.log(nn.Sigmoid()(y_pre1)) + (1-y)*(torch.log(1-nn.Sigmoid()(y_pre1))))
output

tensor(0.5001)

# 多分类交叉熵

In [6]:
# 标签是一维的，有几种不同数字就是分类数目
y = torch.LongTensor([0, 1, 2, 0])
# 行数就是数据个数，列数就是分类数目
y_pre1 = torch.tensor([[1, 0.2, 0.1],
                    [0.1, 1, 0.1],
                    [0.1, 0.2, 1],
                    [1, 0.2, 0.1]])

y_pre2 = torch.tensor([[1, 0.2, 5],
                    [0.1, 1, 5],
                    [3, 0.2, 1],
                    [0.9, 0.2, 0.1]])              
# 每一个数据做softmax处理
m = nn.LogSoftmax(dim=1)
y_pre_li = [m(y_pre1), m(y_pre2)]
# nn.NLLLoss的输入target是类别值
criterion = nn.NLLLoss()
for i in range(2):
    output = criterion(y_pre_li[i], y)
    print('第{}个预测值和实际值的损失值：{}'.format(i+1, output))

第1个预测值和实际值的损失值：0.612541675567627
第2个预测值和实际值的损失值：2.7241179943084717


In [7]:
m(y_pre1)

tensor([[-0.6184, -1.4184, -1.5184],
        [-1.4951, -0.5951, -1.4951],
        [-1.5184, -1.4184, -0.6184],
        [-0.6184, -1.4184, -1.5184]])

# 多分类第二种方式

In [8]:
# 标签，这里注意必须是LongTensor类型
y = torch.LongTensor([0, 1, 2, 0])
# CrossEntropyLoss多分类，用这种方便,这里4行代表4个数据, 3列代表3种分类
y_pre1 = torch.tensor([[1, 0.2, 0.1],
                    [0.1, 1, 0.1],
                    [0.1, 0.2, 1],
                    [1, 0.2, 0.1]])

y_pre2 = torch.tensor([[1, 0.2, 5],
                    [0.1, 1, 5],
                    [3, 0.2, 1],
                    [0.9, 0.2, 0.1]]) 
y_pre_li = [y_pre1, y_pre2]
# 传入预测值的维度就是类别数量（几种分类），第二个是真实值的维度就是一维用的是类别的索引（0表示第一个类别，2表示第三个类别）
criterion = nn.CrossEntropyLoss()
for i in range(2):
    output = criterion(y_pre_li[i], y)
    print('第{}个预测值和实际值的损失值：{}'.format(i+1, output))

第1个预测值和实际值的损失值：0.612541675567627
第2个预测值和实际值的损失值：2.7241179943084717


<font color=#A52A2A size=4 >nn.NLLLoss的输入target是类别值，并不是one-hot编码格式，这个要注意！！  
CrossEntropyLoss()的target输入也是类别值，不是one-hot编码格式</font>

$loss(x,class) = -log(\frac{exp^{x[class]}}{\sum exp^{x[i]}})$

In [9]:
# 算法实现
# 定义softmax函数
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

# 利用numpy计算
def cross_entropy_np(x, y):
    x_softmax = [softmax(x[i]) for i in range(len(x))]
    x_log = [np.log(x_softmax[i][y[i]]) for i in range(len(y))]
    print(x_log)
    loss = - np.sum(x_log) / len(y)
    return loss
cross_entropy_np(y_pre1.numpy(), y.numpy())

[-0.61836904, -0.5950598, -0.61836904, -0.61836904]


0.6125417351722717

In [10]:
# 计算某个数的loss
num = 3
y_pre1.numpy(),y.numpy()

(array([[1. , 0.2, 0.1],
        [0.1, 1. , 0.1],
        [0.1, 0.2, 1. ],
        [1. , 0.2, 0.1]], dtype=float32),
 array([0, 1, 2, 0], dtype=int64))

In [11]:
# c表示这个数的类别，对应就是y这个标签的值
c = y.numpy()[num]
c,np.exp(y_pre1.numpy()[num][c]),np.sum(np.exp(y_pre1.numpy()[num]))

(0, 2.7182817, 5.044855)

In [12]:
np.log(np.exp(y_pre1.numpy()[num][c])/np.sum(np.exp(y_pre1.numpy()[num])))

-0.61836904

BCELoss是Binary CrossEntropyLoss的缩写，BCELoss是CrossEntropyLoss的一个特例，只用于二分类问题，而CrossEntropyLoss可以用于二分类，也可以用于多分类。  
使用nn.BCELoss需要在该层前面加上Sigmoid函数。  
使用nn.CrossEntropyLoss会自动加上Softmax层。  